To do:
a) Structure for part-1: Finding which nodes are important (metrics used is entropy)
- Create a class that can experiment with different: 1)2 datasets, 2)3 seeds, 3)compression rate, 4)Models - GCN, GAT, Gsage, 5) Masks : only high, only low, Mix: 50:50, 20:80, 80:20, highDiverseLow
- For a dataset & seed, 
- a) FindEntropy: Use benchmark model to find entropy of points. Classify into high and low entropy. (STORE : List of (Entropy, nodeNumber) ) --- this will be used in part b
- b) createMasks: creates masks of various types.
- c) TrainModel: For each masks created, it trains model to find (best accuracy) during training.
- d) Output: 1) For each seed create one dataframe. Each Dataframe: Rows - Dataset/Compression, Col - Models/Masks + 2) Create a aggregate dataframe that store average. Store output every few iterations.

b) Structure for part-2: For each method, in which category (high, mid, low) do the synthetic nodes fit in.
- Create class that does the below for a) 2 datasets, b) different distill seeds, c) compression, d) 
- a) Reading the saved (A,X) of distillation methods and converting them into useful data structure for GNN model
- b) Passing through the bench model to find entropy of each node
- c) Assigning the (node, entropy) to high, low, mid : (rank the node based on closness to entropy of the stored list)
- d) Output: 1) Dataframe for each seed and aggregate dataframe for average. Store after every few iterations. Row: Dataset/compression, column: Method/(high,mid,low)

In [1]:
import torch
import os
import matplotlib.pyplot as plt
from sklearn import decomposition
from sklearn.manifold import TSNE
import pandas as pd

import torch_geometric
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch.nn import Linear
from torch_geometric.nn import GATConv
from sklearn.decomposition import PCA
import numpy as np

import random

In [2]:
print(torch.__version__)

2.1.2+cu121


In [3]:
class GCN(torch.nn.Module):
    def __init__(self, input_feature, nclasses):
        super().__init__()
        
        self.input_feature = input_feature
        self.nclasses = nclasses
        #define layers
        self.conv1 = GCNConv(in_channels=self.input_feature,out_channels=1024)
        self.conv2 = GCNConv(in_channels=1024, out_channels=512)
        self.conv3 = GCNConv(in_channels= 512, out_channels=512)
        self.classifier = Linear(512,self.nclasses)

    def forward(self,x, edge_idx):
        a = self.conv1(x,edge_idx)
        a= a.tanh()
        a = self.conv2(a,edge_idx)
        a = a.tanh()
        a = self.conv3(a,edge_idx)
        a= a.tanh()
        out = self.classifier(a)

        return out,a

class GAT(torch.nn.Module):
    def __init__(self, input_feature, nclasses):
        super().__init__()
        
        self.input_feature = input_feature
        self.nclasses = nclasses
        #define layers
        self.conv1 = GATConv(in_channels=self.input_feature,out_channels=1024)
        self.conv2 = GATConv(in_channels=1024, out_channels=512)
        self.conv3 = GATConv(in_channels= 512, out_channels=512)
        self.classifier = Linear(512,self.nclasses)

    def forward(self,x, edge_idx):
        a = self.conv1(x,edge_idx)
        a= a.tanh()
        a = self.conv2(a,edge_idx)
        a = a.tanh()
        a = self.conv3(a,edge_idx)
        a= a.tanh()
        out = self.classifier(a)

        return out,a

In [4]:
def mask_to_index(index, size):
    all_idx = np.arange(size)
    return all_idx[index]

def index_to_mask(index, size):
    mask = torch.zeros((size, ), dtype=torch.bool)
    mask[index] = 1
    return mask

def get_augmented_edges(nclasses, node_per_class):

    # Initialize an empty set to store unique edges
    num_nodes = nclasses*node_per_class
    edges_augmented = set()
    allnodes = [i for i in range(num_nodes)]
    for i in range(nclasses):
        idx = i*node_per_class
        nodes = allnodes[idx:idx+node_per_class]
        combination = [(i,j) for i in nodes for j in nodes if i!=j]
        edges_augmented.update(combination)
        
#     print(edges_augmented)
    out = torch.tensor(list(edges_augmented)).t().contiguous()
    return out

In [6]:
# get_augmented_edges(2, 4)

In [7]:
# class training():
#     def __init__(self, model, data, mask = None):
#         self.model = model
#         self.dataset = data
#         self.criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
#         self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.00012)  # Define optimizer.
#         if mask == None:
#             self.train_old_mask = self.dataset.data.train_mask
#         else:
#             self.train_old_mask = mask
#             self.dataset.data.train_mask = mask
#         self.embed = 0
        
#         #print("---Total training points --- ", )
        
#         self.best_accuracy=0


#     def train(self, data):
#         self.optimizer.zero_grad()  # Clear gradients.
#         out, h = self.model.forward(data.x, data.edge_index)  # Perform a single forward pass.

#         loss = self.criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.

#         loss.backward()  # Derive gradients.
#         self.optimizer.step()  # Update parameters based on gradients.
#         return loss, h, out

#     def train_model(self, epo=200):
        
#         data = self.dataset[0]
#         #print("Shape is', data.x.shape)
#         loss_t = []

#         train_loss = []
#         val_loss = []
#         test_loss = []

#         test_accuracy = []
#         train_accuracy = []
#         val_accuracy = []

#         ep=[]

#         macro_f1_train = []
#         macro_f1_val =[]
#         macro_f1_test=[]

#         print('Training Started...')
#         for epoch in range(1,epo):
#             loss, h, out = self.train(data)
#             loss_t+=[loss]

#             if epoch % 5 == 0:
#                 #visualize_embedding(h, color=data.y, epoch=epoch, loss=loss)
#                 ep.append(epoch)

#                 with torch.no_grad():
                    
#                     train_loss.append(self.criterion(out[data.train_mask], data.y[data.train_mask]).item())
#                     val_loss.append(self.criterion(out[data.val_mask], data.y[data.val_mask]).item())
#                     test_loss.append(self.criterion(out[data.test_mask], data.y[data.test_mask]).item())

#                     train_accuracy.append(((torch.argmax(out[data.train_mask], axis=1) == data.y[data.train_mask]).sum()/len(data.y[data.train_mask])).item()*100)
#                     val_accuracy.append(((torch.argmax(out[data.val_mask], axis=1) == data.y[data.val_mask]).sum()/len(data.y[data.val_mask])).item()*100)
                    
#                     test_acc = ((torch.argmax(out[data.test_mask], axis=1) == data.y[data.test_mask]).sum()/len(data.y[data.test_mask])).item()*100
#                     test_accuracy.append(test_acc)
#                     if test_acc> self.best_accuracy:
#                         self.best_accuracy = test_acc
#                         print("Saving Model at acc: ", self.best_accuracy)
#                         torch.save(self.model.state_dict(), f'Best_model.pt')

#                     print(f'Epoch : {epoch:.2f}, Training Accuracy: {train_accuracy[-1]:.2f}, Testing Accuracy: {test_accuracy[-1]:.2f}')


#         print('Training Finished!')
#         print("Best accuracy: ", self.best_accuracy )
#         with torch.no_grad():
#             self.model.load_state_dict(torch.load('Best_model.pt'))

In [8]:
class train_d():
    def __init__(self, model, data, test_data):
        self.model = model
        self.data = data
        self.test_data = test_data
        self.criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.00012)  # Define optimizer.
        
        self.embed = 0
        self.best_accuracy=0
        
        

    def train(self, data):
        self.optimizer.zero_grad()  # Clear gradients.
        out, h = self.model.forward(data.x, data.edge_index)  # Perform a single forward pass.

        loss = self.criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.

        loss.backward()  # Derive gradients.
        self.optimizer.step()  # Update parameters based on gradients.
        return loss, h, out
    

    def train_model(self, epo = 200):
        data = self.data
        print('Training Started...')
        train_accuracy = []
        test_accuracy = []
        for epoch in range(1,epo):
            loss, h, out = self.train(data)
#             loss_t+=[loss]

            if epoch % 5 == 0:
                #visualize_embedding(h, color=data.y, epoch=epoch, loss=loss)
#                 ep.append(epoch)

                with torch.no_grad():

                    train_accuracy.append(((torch.argmax(out[data.train_mask], axis=1) == data.y[data.train_mask]).sum()/len(data.y[data.train_mask])).item()*100)
                    
                    test_acc = self.test()   # test with the original test data
                    test_accuracy.append(test_acc)
                    if test_acc> self.best_accuracy:
                        self.best_accuracy = test_acc
                        print("Saving Model at acc: ", self.best_accuracy)
                        torch.save(self.model.state_dict(), f'Best_model.pt')

                    print(f'Epoch : {epoch:.2f}, Training Accuracy: {train_accuracy[-1]:.2f}, Testing Accuracy: {test_accuracy[-1]:.2f}')


        print('Training Finished!')
        print("Best accuracy: ", self.best_accuracy )
        with torch.no_grad():
            self.model.load_state_dict(torch.load('Best_model.pt'))
    def test(self):
        data = self.test_data
        with torch.no_grad():
            out, h = self.model.forward(data.x, data.edge_index)
            acc = ((torch.argmax(out[data.test_mask], axis=1) == data.y[data.test_mask]).sum()/len(data.y[data.test_mask])).item()*100
            return acc

In [23]:
class experiment3():
    def __init__(self,data_list, seedlist, compress_rate_list, models_list):
        # bench models - dictionary : {(dataname,modeltype,seed):model}
        self.data_list = data_list  #[cora, citeseer]
        self.seed_list = seedlist   #[1,85,105]
        self.compress_rate_list = compress_rate_list  #[0.5,0.3]
        self.model_list = models_list      #["GCN", "GAT"]        
        
        # ready the dataset
        self.datasets = {}                # {dataname:data}
        for dataname in data_list:
            self.datasets[dataname]= self.get_dataset(dataname)
        
        self.entropy = {}
        # creat output table
        self.mask_list = ["High-only", "Low-only", "Mix-0.5", "Mix-0.3H", "Mix-0.7H", "Mix-Div"]
        self.store= datastorage(self.model_list, self.mask_list, self.compress_rate_list)
        
        # ready bench models:
#         for dataname in self.data_list:
#             dataset = self.datasets[dataname]
#             data = dataset[0]
#             for seed in self.seed_list:
#                 for modeltype in self.model_list:
#                     if modeltype =="GCN":
#                         model = GCN(data.num_features,data.num_classes)
#                     elif modeltype == "GAT":
#                         model = GAT(data.num_features,data.num_classes)
#                     model.load_state_dict(torch.load(f'Bench_model_{dataname}_{modeltype}_{seed}.pt'))
#                     self.bench_models[(dataname,modeltype,seed)] = model
        
    
    def run_experiment(self):
        for seed in self.seed_list:
            print("**********************************************************")
            random.seed(seed)
            np.random.seed(seed)
            torch.manual_seed(seed)
            torch.cuda.manual_seed(seed)
        
            # for a modeltype and dataname, Find entropy
            for dataname in self.data_list:
                #all_models = self.bench_models[dataname]
                print("################NEW Data################ ")
                dataset = self.datasets[dataname]
                data = dataset[0]
#                 original_train_mask = data.train_mask.copy()

                for modeltype in self.model_list:
                    #model = self.bench_models[(dataname,modeltype,seed)]
                    print(f"==== SEED = {seed}, Data = {dataname}, MODEL TYPE = {modeltype} =============")
                    if modeltype =="GCN":
                        model = GCN(dataset.num_features,dataset.num_classes)
                    elif modeltype == "GAT":
                        model = GAT(dataset.num_features,dataset.num_classes)
                    mtype_lower = modeltype.lower()
                    model.load_state_dict(torch.load(f'Bench_model_{dataname}_{mtype_lower}_{seed}.pt'))
                    # find entropy
                    

                    for compression in self.compress_rate_list:
                        print(f"==== compression = {compression} =============")
                        entropy_data,nprob_train = self.find_entropy(dataname,model)
                        n = sum(data.y[data.train_mask] == 0) # number of points in training set per class
#                         print(n)
                        train_pts = int(compression*n)      # cora and citeseer has 20 training points each
#                         print(train_pts)
#                         print("Shape of nprob",nprob_train.shape, "Shape of Entropy", entropy_data.shape, "train pt", train_pts )

                        _, masks = self.allmasks(nprob_train,entropy_data,data, train_pts)  # {high:masks, low:mask}
                        
#                         original_train_mask = data.train_mask.clone()
                        for masktype, mask in masks.items():
                            idxlist = mask_to_index(mask, len(mask))
                            print("===========New Mask=================")
                            print("Mask type:", masktype, "num of points", len(idxlist) )
                            print("Mask Element: ", idxlist )
            
                            if modeltype =="GCN":
                                model_new = GCN(dataset.num_features,dataset.num_classes)
                            elif modeltype == "GAT":
                                model_new = GAT(dataset.num_features,dataset.num_classes)

                            new_graph = self.get_graph(idxlist, data, train_pts, dataset.num_classes, augment= False)
#                             asdfasdfs = popoasdfas
                            exp_new = train_d(model_new, new_graph, data)
                            exp_new.train_model()
                            result = exp_new.best_accuracy
                            # store the results
                            self.store.update(result, [dataname, compression, modeltype, masktype, seed])
                            
                        
#                         self.datasets[dataname].data.train_mask = original_train_mask
                        
                            
    def get_graph(self, selected_idx, data, nod_per_class, nclasses, augment=False):
        # new features
#         print("idx: ", selected_idx)
#         print("data.y", data.y)
#         print("next", data.x[selected_idx] )
        newx = data.x[selected_idx]
        
        if augment == True:
            new_edges = get_augmented_edges(nclasses, nod_per_class) # classes, nodes per class
        else:
        
            # new edges_list
            edge_mask = torch.zeros(data.edge_index.size(1), dtype=torch.bool)

            selected_nodes_set = set(selected_idx.tolist())
            edge_mask = (torch.tensor([i.item() in selected_nodes_set for i in data.edge_index[0]]) & torch.tensor([i.item() in selected_nodes_set for i in data.edge_index[1]])).bool()
            edge_mask = torch.where(edge_mask, torch.tensor([1], dtype=torch.bool), torch.tensor([0], dtype=torch.bool))

            new_edges = data.edge_index[:, edge_mask]

            # Remap node indices in the new edge list
            mapping = {old_idx: new_idx for new_idx, old_idx in enumerate(selected_idx)}
            new_edges = torch.tensor([[mapping[old_idx.item()] for old_idx in new_edges[0]],[mapping[old_idx.item()] for old_idx in new_edges[1]]], dtype=torch.long)

        print("Old vs New Edges shape", data.edge_index.shape, new_edges.shape)
        # new label
        labels_new = data.y[selected_idx]
        
        return Data(x=newx, edge_index=new_edges, y=labels_new, train_mask=[True]*len(labels_new))
    
    def find_entropy(self,dataname,model):
        dataset = self.datasets[dataname]
        
        # make model fixed and extract dat
        model.eval()
        data= dataset[0]
        print("-- at Find Entropy, datamask shape", data.train_mask.shape)
        # need a forward pass to find score and embed
        score, embed = model.forward(data.x, data.edge_index)
        
        # need to apply softmax to convert scores to probability
        runsoftmax = torch.nn.Softmax(dim=1)
        nprob = runsoftmax(score)
        trainmask = data.train_mask
        nprob_train = nprob[trainmask]
        
        # Calculate entropy for each data point
        entropy_data = -torch.sum(nprob_train * torch.log2(nprob_train + 1e-10), dim=1)   #1e-10 so that there are no very small numbers
        return entropy_data,nprob_train
        
    def allmasks(self,nprob_train,entropy_data, data, sample_size):
        trainmask = data.train_mask
              
        out = {"High-only":[], "Low-only":[], "Mix-0.5":[], "Mix-0.3H":[], "Mix-0.7H":[], "Mix-Div":[]}
        out_indices = {"High-only":[], "Low-only":[], "Mix-0.5":[], "Mix-0.3H":[], "Mix-0.7H":[], "Mix-Div":[]}
#         print(sample_size)
        highidx, lowidx, highmask, lowmask, fullmask = self.find_new_mask(nprob_train, trainmask,entropy_data,data, nsamples=sample_size)
        out["High-only"]= highmask
        out["Low-only"] = lowmask
        out_indices["High-only"] = highidx
        out_indices["Low-only"] = lowidx
        
        half_h_idx, half_l_idx, half_high, half_low, halfm = self.find_new_mask(nprob_train, trainmask,entropy_data,data, nsamples=sample_size//2)
        out["Mix-0.5"] = torch.logical_or(half_high,half_low)
        out_indices["Mix-0.5"] = torch.cat((half_h_idx, half_l_idx), dim=0)
        
        a= int(round(sample_size*0.3,0))
        b = sample_size-a
        ahighidx, alowidx, ahigh, alow, _ = self.find_new_mask(nprob_train, trainmask,entropy_data, data, nsamples=a)
        bhighidx, blowidx, bhigh, blow, _ = self.find_new_mask(nprob_train, trainmask,entropy_data, data, nsamples=b)
        out["Mix-0.3H"] = torch.logical_or(ahigh,blow)
        out["Mix-0.7H"] = torch.logical_or(bhigh,alow)
        out_indices["Mix-0.3H"] = torch.cat((ahighidx,blowidx),dim=0)
        out_indices["Mix-0.7H"] = torch.cat((bhighidx,alowidx))
        
        # for top k high and diverse k low (50:50)
        k= sample_size//2
        n = sum(data.y[trainmask] == 0)  # any class will do
        n = int(n.item()//2)        # half is low entropy and other half is high entropy
        jumps = n//k
        idxlist = [i for i in range(1, n+1) if i%jumps==0]
        khighidx,klowidx, khigh, klow, _ = self.find_new_mask(nprob_train, trainmask,entropy_data, data, nsamples=k)
        dmask = khigh
        dmask_idx = khighidx
        for idx in idxlist:
            ihighidx, ilowidx, ihigh, ilow, _ = self.find_new_mask(nprob_train, trainmask,entropy_data, data, nsamples=idx)
            jhighidx, jlowidx, jhigh, jlow,_ = self.find_new_mask(nprob_train, trainmask,entropy_data, data, nsamples=idx-1)
            dmask = torch.logical_or(dmask,torch.logical_xor(ilow,jlow))
#             dmask_idx = torch.cat((dmask_idx, torch.setdiff1d(ilowidx, jlowidx)))
            dmask_idx = torch.cat((dmask_idx, torch.tensor(list(set(ilowidx.numpy()) - set(jlowidx.numpy())))))

        
        out["Mix-Div"] = dmask
        out_indices["Mix-Div"] = dmask_idx
        
        return out_indices, out
        
    
    def find_new_mask(self, nprob_train, trainmask,entropy_data, data, nsamples=1):
        
        
        # Create new train masks with only highest and lowest entropy points set to True
        new_high_train_mask = torch.zeros_like(trainmask)
        new_low_train_mask = torch.zeros_like(trainmask)
        high_idx = torch.empty((0),dtype=torch.int32)
        low_idx = torch.empty((0),dtype=torch.int32)
        predicted_classes = torch.argmax(nprob_train, dim=1)
        
        # Iterate over each class
        for class_id in range(nprob_train.shape[1]):
            
            class_mask = (data.y[trainmask] == class_id) & (predicted_classes==data.y[trainmask])
#             print(class_id, " -- inside ", entropy_data.shape, class_mask.shape)
            class_entropies = entropy_data[class_mask]

            # Find indices of highest and lowest entropies for the current class
            highest_indices = torch.argsort(class_entropies, descending=True)[:nsamples]
            lowest_indices = torch.argsort(class_entropies)[:nsamples]
#             print(highest_indices, len(new_high_train_mask[trainmask][data.y[trainmask] == class_id][highest_indices]))
#             print(lowest_indices, len(new_low_train_mask[trainmask][data.y[trainmask] == class_id][lowest_indices]))
            high_idx = torch.cat((high_idx,highest_indices))
            low_idx = torch.cat((low_idx,lowest_indices))

            new_high_indices = torch.nonzero(class_mask)[highest_indices]  # Indices in the original tensor
            new_low_indices = torch.nonzero(class_mask)[lowest_indices]  # Indices in the original tensor

            new_high_train_mask[new_high_indices] = 1
            new_low_train_mask[new_low_indices] = 1

            full_mask = torch.logical_or(new_low_train_mask, new_high_train_mask)
#         print(class_entropies)
        return high_idx, low_idx, new_high_train_mask, new_low_train_mask, full_mask
    
    
                        
    def get_dataset(self,dataname):
        data = Planetoid(root='data', name= dataname)
        return data
        

In [10]:
class datastorage():
    def __init__(self, model_list, mask_list, compression_list):
        self.models = model_list
        self.masks = mask_list
        self.data = ["cora", "citeseer"]
        self.compression = compression_list # [0.25,0.5]

        # create tables 3seeds - 1,15,85 
        self.results_1 = self.create_table()
        self.results_105 = self.create_table()
        self.results_85 = self.create_table()
        
        self.i =0

    def create_table(self):
        
        col_idx = pd.MultiIndex.from_product([self.models,self.masks],names = ["Models","Masks"])
        row_idx = pd.MultiIndex.from_product([self.data,self.compression],names = ["Dataset","Compression ratio"])
        df = pd.DataFrame(columns=col_idx, index=row_idx)
        df = df.fillna(0)
        return df

    def update(self, results, details):
        
        dataset, compression, model, mask, seed = details

        if seed==1:
            self.results_1.loc[(dataset,compression),(model,mask)] = results

        if seed==105:
             self.results_105.loc[(dataset,compression),(model,mask)] = results

        if seed == 85:
            self.results_85.loc[(dataset,compression),(model,mask)] = results
            
        self.i+=1
        if self.i%5==0:
            self.display_save()


    def display_save(self):
        # Create a new dataframe 'average' for jacard nodes
        self.average_results = self.create_table()
        for col in self.average_results.columns:
            self.average_results[col] = (self.results_1[col] + self.results_105[col] + self.results_85[col]) / 3

        # print("Jacard Nodes Similarity")
        # print(average_jac_nodes)
        self.average_results.to_csv("Avg_exp2_partA_results-jan_aug.csv")

        # save all seeds files
        output_folder = 'rawFiles'
        os.makedirs(output_folder, exist_ok=True)
        self.results_1.to_csv("rawFiles/results_1-jan_aug.csv")
        self.results_105.to_csv("rawFiles/results_105-jan_aug.csv")
        self.results_85.to_csv("rawFiles/results_85-jan_aug.csv")


In [11]:
class create_bench():
    def __init__(self, seedlist, datalist):
        self.seedlist = seedlist
        self.datalist = datalist
        # ready the dataset
        self.datasets = {}                # {dataname:data}
        for dataname in self.datalist:
            self.datasets[dataname]= self.get_dataset(dataname)
            
        self.out = {}
        
        
    def bench_model_all(self):
        for seed in self.seedlist:
            #set seed
            random.seed(seed)
            np.random.seed(seed)
            torch.manual_seed(seed)
            torch.cuda.manual_seed(seed)
            
            for dataname, dataset in self.datasets.items():
                data = dataset[0]
                model_gcn = GCN(dataset.num_features,dataset.num_classes)
                model_gat = GAT(dataset.num_features,dataset.num_classes)
                
                # train GCN
                exp = training(model_gcn, dataset)
                exp.train_model()
                model = exp.model
                torch.save(model.state_dict(), f'Bench_model_{dataname}_gcn_{seed}.pt')
                self.out[(dataname,"gcn",seed)] = model
                
                # train GAT
                exp1 = training(model_gat, dataset)
                exp1.train_model()
                model = exp1.model
                torch.save(model.state_dict(), f'Bench_model_{dataname}_gat_{seed}.pt')
                self.out[(dataname,"gat",seed)] = model
                
                 
        
    def get_dataset(self,dataname):
        data = Planetoid(root='data', name= dataname)
        return data
    

## MAIN: Creating Benchmark models

In [ ]:
seedlist = [1,85,105]
datalist = ["cora", "citeseer"]
bench = create_bench(seedlist, datalist)
bench.datasets

In [ ]:
# bench.bench_model_all()

In [24]:
data_list = ["cora", "citeseer"]
seedlist = [1,85,105] 
compress_rate_list= [0.5,0.3]
models_list = ["GCN", "GAT"]
exp = experiment3(data_list,seedlist,compress_rate_list, models_list)

In [25]:
exp.run_experiment()

**********************************************************
################NEW Data################ 
==== SEED = 1, Data = cora, MODEL TYPE = GCN =============
==== compression = 0.5 =============
-- at Find Entropy, datamask shape torch.Size([2708])
===========New Mask=================
Mask type: High-only num of points 70
Mask Element:  [  2   4   8   9  11  12  13  14  21  22  23  27  28  29  30  32  34  35
  37  38  41  45  46  48  49  50  53  54  56  57  58  60  61  65  66  68
  69  70  72  76  77  79  82  84  85  86  88  90  93  94  96  97 100 101
 102 103 108 112 114 115 116 118 120 124 126 129 130 131 136 138]
Old vs New Edges shape torch.Size([2, 10556]) torch.Size([2, 6])
Training Started...
Saving Model at acc:  38.199999928474426
Epoch : 5.00, Training Accuracy: 85.71, Testing Accuracy: 38.20
Saving Model at acc:  53.60000133514404
Epoch : 10.00, Training Accuracy: 94.29, Testing Accuracy: 53.60
Saving Model at acc:  61.10000014305115
Epoch : 15.00, Training Accuracy: 95.71

Epoch : 55.00, Training Accuracy: 98.57, Testing Accuracy: 64.70
Epoch : 60.00, Training Accuracy: 98.57, Testing Accuracy: 63.70
Epoch : 65.00, Training Accuracy: 98.57, Testing Accuracy: 62.80
Epoch : 70.00, Training Accuracy: 98.57, Testing Accuracy: 62.40
Epoch : 75.00, Training Accuracy: 98.57, Testing Accuracy: 61.90
Epoch : 80.00, Training Accuracy: 98.57, Testing Accuracy: 61.50
Epoch : 85.00, Training Accuracy: 98.57, Testing Accuracy: 61.00
Epoch : 90.00, Training Accuracy: 98.57, Testing Accuracy: 61.10
Epoch : 95.00, Training Accuracy: 98.57, Testing Accuracy: 60.80
Epoch : 100.00, Training Accuracy: 98.57, Testing Accuracy: 60.70
Epoch : 105.00, Training Accuracy: 98.57, Testing Accuracy: 60.70
Epoch : 110.00, Training Accuracy: 98.57, Testing Accuracy: 60.60
Epoch : 115.00, Training Accuracy: 98.57, Testing Accuracy: 60.60
Epoch : 120.00, Training Accuracy: 98.57, Testing Accuracy: 60.60
Epoch : 125.00, Training Accuracy: 98.57, Testing Accuracy: 60.50
Epoch : 130.00, Tra

Epoch : 180.00, Training Accuracy: 98.57, Testing Accuracy: 63.10
Epoch : 185.00, Training Accuracy: 98.57, Testing Accuracy: 63.20
Epoch : 190.00, Training Accuracy: 98.57, Testing Accuracy: 63.50
Epoch : 195.00, Training Accuracy: 98.57, Testing Accuracy: 63.50
Training Finished!
Best accuracy:  68.59999895095825
===========New Mask=================
Mask type: Mix-Div num of points 70
Mask Element:  [  0   1   4   6  11  12  13  15  16  17  20  21  22  23  25  26  29  30
  31  32  33  34  36  37  38  40  41  42  49  50  53  54  57  59  60  61
  62  63  67  68  69  71  72  74  75  76  79  80  81  83  86  93 101 104
 110 112 113 115 116 120 121 122 127 128 131 132 134 136 137 138]
Old vs New Edges shape torch.Size([2, 10556]) torch.Size([2, 4])
Training Started...
Saving Model at acc:  40.09999930858612
Epoch : 5.00, Training Accuracy: 87.14, Testing Accuracy: 40.10
Saving Model at acc:  61.400002241134644
Epoch : 10.00, Training Accuracy: 95.71, Testing Accuracy: 61.40
Saving Model at

Epoch : 70.00, Training Accuracy: 100.00, Testing Accuracy: 61.60
Epoch : 75.00, Training Accuracy: 100.00, Testing Accuracy: 61.60
Epoch : 80.00, Training Accuracy: 100.00, Testing Accuracy: 61.20
Epoch : 85.00, Training Accuracy: 100.00, Testing Accuracy: 60.90
Epoch : 90.00, Training Accuracy: 100.00, Testing Accuracy: 61.10
Epoch : 95.00, Training Accuracy: 100.00, Testing Accuracy: 61.30
Epoch : 100.00, Training Accuracy: 100.00, Testing Accuracy: 61.40
Epoch : 105.00, Training Accuracy: 100.00, Testing Accuracy: 61.40
Epoch : 110.00, Training Accuracy: 100.00, Testing Accuracy: 61.70
Epoch : 115.00, Training Accuracy: 100.00, Testing Accuracy: 61.90
Epoch : 120.00, Training Accuracy: 100.00, Testing Accuracy: 62.00
Epoch : 125.00, Training Accuracy: 100.00, Testing Accuracy: 62.00
Epoch : 130.00, Training Accuracy: 100.00, Testing Accuracy: 62.00
Epoch : 135.00, Training Accuracy: 100.00, Testing Accuracy: 62.20
Epoch : 140.00, Training Accuracy: 100.00, Testing Accuracy: 62.30
E

Old vs New Edges shape torch.Size([2, 10556]) torch.Size([2, 4])
Training Started...
Saving Model at acc:  43.299999833106995
Epoch : 5.00, Training Accuracy: 83.33, Testing Accuracy: 43.30
Saving Model at acc:  51.70000195503235
Epoch : 10.00, Training Accuracy: 92.86, Testing Accuracy: 51.70
Saving Model at acc:  57.099997997283936
Epoch : 15.00, Training Accuracy: 97.62, Testing Accuracy: 57.10
Saving Model at acc:  58.09999704360962
Epoch : 20.00, Training Accuracy: 97.62, Testing Accuracy: 58.10
Saving Model at acc:  59.20000076293945
Epoch : 25.00, Training Accuracy: 97.62, Testing Accuracy: 59.20
Epoch : 30.00, Training Accuracy: 97.62, Testing Accuracy: 58.90
Epoch : 35.00, Training Accuracy: 97.62, Testing Accuracy: 58.40
Epoch : 40.00, Training Accuracy: 97.62, Testing Accuracy: 57.10
Epoch : 45.00, Training Accuracy: 97.62, Testing Accuracy: 55.90
Epoch : 50.00, Training Accuracy: 97.62, Testing Accuracy: 55.40
Epoch : 55.00, Training Accuracy: 97.62, Testing Accuracy: 54.20

...

===========New Mask=================
Mask type: Mix-0.5 num of points 60
Mask Element:  [  3   5   8  11  12  13  14  16  17  21  25  28  34  35  39  40  41  42
  45  46  47  48  50  52  53  54  55  56  58  62  63  64  65  67  71  72
  75  78  82  83  85  87  89  90  91  93  94  97  98 100 101 105 106 109
 111 112 114 115 116 117]
Old vs New Edges shape torch.Size([2, 9104]) torch.Size([2, 4])
Training Started...
Saving Model at acc:  41.600000858306885
Epoch : 5.00, Training Accuracy: 100.00, Testing Accuracy: 41.60
Saving Model at acc:  55.900001525878906
Epoch : 10.00, Training Accuracy: 100.00, Testing Accuracy: 55.90
Saving Model at acc:  58.60000252723694
Epoch : 15.00, Training Accuracy: 100.00, Testing Accuracy: 58.60
Saving Model at acc:  59.299999475479126
Epoch : 20.00, Training Accuracy: 100.00, Testing Accuracy: 59.30
Saving Model at acc:  59.3999981880188
Epoch : 25.00, Training Accuracy: 100.00, Testing Accuracy: 59.40
Epoch : 30.00, Training Accuracy: 100.00, Testing Ac

Epoch : 60.00, Training Accuracy: 100.00, Testing Accuracy: 62.90
Epoch : 65.00, Training Accuracy: 100.00, Testing Accuracy: 63.10
Epoch : 70.00, Training Accuracy: 100.00, Testing Accuracy: 63.30
Epoch : 75.00, Training Accuracy: 100.00, Testing Accuracy: 63.30
Epoch : 80.00, Training Accuracy: 100.00, Testing Accuracy: 63.10
Epoch : 85.00, Training Accuracy: 100.00, Testing Accuracy: 63.20
Epoch : 90.00, Training Accuracy: 100.00, Testing Accuracy: 63.20
Epoch : 95.00, Training Accuracy: 100.00, Testing Accuracy: 63.20
Epoch : 100.00, Training Accuracy: 100.00, Testing Accuracy: 63.20
Epoch : 105.00, Training Accuracy: 100.00, Testing Accuracy: 63.30
Epoch : 110.00, Training Accuracy: 100.00, Testing Accuracy: 63.30
Epoch : 115.00, Training Accuracy: 100.00, Testing Accuracy: 63.30
Epoch : 120.00, Training Accuracy: 100.00, Testing Accuracy: 63.20
Epoch : 125.00, Training Accuracy: 100.00, Testing Accuracy: 63.10
Epoch : 130.00, Training Accuracy: 100.00, Testing Accuracy: 63.10
Epo

Epoch : 180.00, Training Accuracy: 100.00, Testing Accuracy: 56.30
Epoch : 185.00, Training Accuracy: 100.00, Testing Accuracy: 56.30
Epoch : 190.00, Training Accuracy: 100.00, Testing Accuracy: 56.30
Epoch : 195.00, Training Accuracy: 100.00, Testing Accuracy: 56.30
Training Finished!
Best accuracy:  56.40000104904175
===========New Mask=================
Mask type: Low-only num of points 36
Mask Element:  [  3   7  11  16  17  21  34  35  40  44  45  46  53  56  58  59  62  63
  64  66  67  71  72  75  82  85  86  89  93  94 103 105 111 112 116 117]
Old vs New Edges shape torch.Size([2, 9104]) torch.Size([2, 4])
Training Started...
Saving Model at acc:  39.80000019073486
Epoch : 5.00, Training Accuracy: 100.00, Testing Accuracy: 39.80
Saving Model at acc:  51.899999380111694
Epoch : 10.00, Training Accuracy: 100.00, Testing Accuracy: 51.90
Saving Model at acc:  54.6999990940094
Epoch : 15.00, Training Accuracy: 100.00, Testing Accuracy: 54.70
Saving Model at acc:  56.199997663497925
E

Epoch : 55.00, Training Accuracy: 100.00, Testing Accuracy: 56.40
Epoch : 60.00, Training Accuracy: 100.00, Testing Accuracy: 56.40
Epoch : 65.00, Training Accuracy: 100.00, Testing Accuracy: 56.60
Epoch : 70.00, Training Accuracy: 100.00, Testing Accuracy: 56.80
Epoch : 75.00, Training Accuracy: 100.00, Testing Accuracy: 57.50
Epoch : 80.00, Training Accuracy: 100.00, Testing Accuracy: 57.70
Epoch : 85.00, Training Accuracy: 100.00, Testing Accuracy: 57.60
Epoch : 90.00, Training Accuracy: 100.00, Testing Accuracy: 57.80
Epoch : 95.00, Training Accuracy: 100.00, Testing Accuracy: 57.90
Epoch : 100.00, Training Accuracy: 100.00, Testing Accuracy: 58.20
Epoch : 105.00, Training Accuracy: 100.00, Testing Accuracy: 58.20
Epoch : 110.00, Training Accuracy: 100.00, Testing Accuracy: 58.40
Epoch : 115.00, Training Accuracy: 100.00, Testing Accuracy: 58.30
Epoch : 120.00, Training Accuracy: 100.00, Testing Accuracy: 58.20
Epoch : 125.00, Training Accuracy: 100.00, Testing Accuracy: 58.30
Epoc

Epoch : 180.00, Training Accuracy: 100.00, Testing Accuracy: 60.50
Epoch : 185.00, Training Accuracy: 100.00, Testing Accuracy: 60.50
Epoch : 190.00, Training Accuracy: 100.00, Testing Accuracy: 60.50
Epoch : 195.00, Training Accuracy: 100.00, Testing Accuracy: 60.50
Training Finished!
Best accuracy:  60.69999933242798


In [ ]:
# # bench models - dictionary : {(dataname,modeltype,seed):model}
#         self.data_list = data_list  #[cora, citeseer]
#         self.seed_list = seedlist   #[1,85,105]
#         self.compress_rate_list = compress_rate_list  #[0.5,0.3]
#         self.model_list = models_list      #["GCN", "GAT"] 

In [ ]:
exp.store.results_1

In [26]:
exp.store.display_save()  # important to run to save the right results and generate correct average

In [27]:
exp.store.average_results

Models                            GCN                                   \
Masks                       High-only   Low-only    Mix-0.5   Mix-0.3H   
Dataset  Compression ratio                                               
cora     0.5                65.433331  74.833333  69.266665  66.999998   
         0.3                52.799998  68.799998  53.566666  62.699999   
citeseer 0.5                59.533334  65.533332  63.233334  64.466667   
         0.3                56.333331  61.533334  54.533333  54.633333   

Models                                                  GAT             \
Masks                        Mix-0.7H    Mix-Div  High-only   Low-only   
Dataset  Compression ratio                                               
cora     0.5                68.700000  70.133334  68.866666  71.999999   
         0.3                55.700000  58.733334  52.366666  64.900001   
citeseer 0.5                62.200000  61.433333  62.266668  62.366666   
         0.3                56.133334  54.866666  56.733334  58.399999   

Models                                                                  
Masks                         Mix-0.5   Mix-0.3H   Mix-0.7H    Mix-Div  
Dataset  Compression ratio                                              
cora     0.5                68.333334  70.000001  68.566666  68.466665  
         0.3                56.833335  56.633333  62.566668  59.800001  
citeseer 0.5                63.799999  62.733334  62.666668  63.933335  
         0.3                56.166667  55.966667  55.866667  59.600000

In [ ]:
exp.store.results_85

In [ ]:
exp.store.results_105